# **Análisis exploratorio de los datos**


The dataset contains transactions made by credit cards in September 2013 by European cardholders.

This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

In [56]:
# Importo librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuro estilo de los gráficos
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## **Carga del dataset situado en PostgreSQL**

In [57]:
import psycopg2

# Datos de conexión a la base de datos
conn = psycopg2.connect(
    host="localhost",
    database="fraude_db",
    user="fraude_user",
    password="0000"
)

# Ejecuto la consulta y cargo el resultado en un DataFrame
df = pd.read_sql("SELECT * FROM creditcard_transactions;", conn)
conn.close()

C:\Users\Ruben\AppData\Local\Temp\ipykernel_5644\3701760687.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM creditcard_transactions;", conn)
